In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import preprocessing

In [2]:
#load and label the data
df = pd.read_csv('../data/city.csv', low_memory=False)
df['assessor_id'] = df['assessor_id'].str[1:]
df = preprocessing.add_labels(df)

In [3]:
df.shape

(18406, 239)

In [4]:
clean = preprocessing.Preprocessing()

In [5]:
df = clean.fit_transform(df)

In [6]:
df.shape

(6564, 81)

In [7]:
df.isnull().sum().sum() #was 32,986

0

In [19]:
neg = df['labels'].value_counts()[0]

In [20]:
pos = df['labels'].value_counts()[1]

In [21]:
pos

1631

In [22]:
pos / (df.shape[0])

0.24847653869591713